In [1]:
import pymesh
import numpy as np
import random
import bisect
import math
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
#load obj files
tea_mesh = pymesh.load_mesh("teapot.obj")
violin_mesh = pymesh.load_mesh("violin_case.obj")

In [3]:
#get vertices and faces
tea_vertices = tea_mesh.vertices
tea_faces = tea_mesh.faces
violin_vertices = violin_mesh.vertices
violin_faces = violin_mesh.faces

In [4]:
#sample points from triangles uniformly
def samplePointsUniformly(area_cumu, faces, vertices):
    P = np.zeros(shape=(10000, 3))
    for i in range(0, 10000):
        r = random.uniform(0, area_cumu[len(area_cumu) - 1])
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        r3 = random.uniform(0, 1)
        tri_index = bisect.bisect_left(area_cumu, r)
        tri = faces[tri_index]
        A = vertices[tri[0]]
        B = vertices[tri[1]]
        C = vertices[tri[2]]
        p = (1 - math.sqrt(r1)) * A + math.sqrt(r1) * (1 - r2) * B + math.sqrt(r1) * r2 * C
        P[i] = p
    return P

In [5]:
#furthest point sampling
def total_distances(p, pts):
    return ((p - pts)**2).sum(axis=1)

def furthestPointSampling(pts, k):
    furthest_pts = np.zeros((k, 3))
    furthest_pts[0] = pts[np.random.randint(len(pts))]
    distances = total_distances(furthest_pts[0], pts)
    for i in range(1, k):
        furthest_pts[i] = pts[np.argmax(distances)]
        distances = np.minimum(distances, total_distances(furthest_pts[i], pts))
    return furthest_pts

In [6]:
# P1 stores 10000 points sampled from the teapot
# D1 is the distance matrix of P1
# S1 stores 1000 points sampled from P1
tea_mesh.add_attribute("face_area")
tea_face_areas = tea_mesh.get_attribute("face_area")
tea_face_areas_cumulative = []
total_area = 0
for i in range(len(tea_face_areas)):
    total_area = total_area + tea_face_areas[i]
    tea_face_areas_cumulative.append(total_area)
P1 = samplePointsUniformly(tea_face_areas_cumulative, tea_faces, tea_vertices)
D1 = pairwise_distances(P1)
S1 = furthestPointSampling(P1, 1000)

In [7]:
# P2 stores 10000 points sampled from the violin case
# D2 is the distance matrix of P2
# S2 stores 1000 points sampled from P2
violin_mesh.add_attribute("face_area")
violin_face_areas = violin_mesh.get_attribute("face_area")
violin_face_areas_cumulative = []
total_area = 0
for i in range(len(violin_face_areas)):
    total_area = total_area + violin_face_areas[i]
    violin_face_areas_cumulative.append(total_area)
P2 = samplePointsUniformly(violin_face_areas_cumulative, violin_faces, violin_vertices)
D2 = pairwise_distances(P2)
S2 = furthestPointSampling(P2, 1000)

In [8]:
#save the S1 to new_teapot.obj
newTri = pymesh.tetgen()
newTri.points = S1
newTri.run()
new_tea = newTri.mesh
pymesh.save_mesh("new_teapot.obj", new_tea, use_float=True)

In [9]:
#save the S2 to new_violin_case.obj
newTri = pymesh.tetgen()
newTri.points = S2
newTri.run()
new_violin = newTri.mesh
pymesh.save_mesh("new_violin_case.obj", new_violin, use_float=True)

In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xs = [s[0] for s in S1]
ys = [s[1] for s in S1]
zs = [s[2] for s in S1]
ax.scatter(xs, ys, zs, c='r', marker='o')

ModuleNotFoundError: No module named 'matplotlib'

In [16]:
#Q3
from sklearn.metrics.pairwise import euclidean_distances

In [22]:
d_origin = euclidean_distances(S1, S2)
d = d_origin

In [20]:
#step 1 row reduction

In [21]:
#step 2 column reduction